In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.
'C:\Users\Andy' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import numpy as np
import datetime
import time
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, balanced_accuracy_score
import pydotplus
from IPython.display import Image
%matplotlib inline
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.combine import SMOTEENN
from imblearn.ensemble import BalancedRandomForestClassifier

In [3]:
ticker = 'SPG'
start_date = '2011-01-01'
end_date = '2021-05-31'

stock_df = yf.download(ticker, start=start_date, end=end_date)
stock_df['Daily Return'] = stock_df['Close'].pct_change().dropna().shift()
stock_df['Daily Rolling Volatility'] = stock_df['Daily Return'].rolling(window=10).std()
stock_df['Decision'] = np.where(stock_df['Daily Return'] > 0, 'Entry' , 'Exit')
stock_df.dropna(inplace=True)
stock_df.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
stock_df.tail(20)

[*********************100%***********************]  1 of 1 completed


,Close,Daily Return,Daily Rolling Volatility,Decision
Date,,,,
2021-05-03,123.199997,-0.012892,0.017750,Exit
2021-05-04,121.360001,0.011993,0.017619,Entry
2021-05-05,120.919998,-0.014935,0.016900,Exit
2021-05-06,122.919998,-0.003626,0.014956,Exit
2021-05-07,124.940002,0.016540,0.013747,Entry
2021-05-10,126.750000,0.016433,0.014062,Entry
2021-05-11,122.639999,0.014487,0.012377,Entry
2021-05-12,117.620003,-0.032426,0.016296,Exit
2021-05-13,117.989998,-0.040933,0.020659,Exit


In [4]:
X = stock_df.copy()
X.drop(['Decision', 'Daily Return'], axis=1, inplace=True)
X.dropna(inplace=True)
X.tail(20)

,Close,Daily Rolling Volatility
Date,,
2021-05-03,123.199997,0.017750
2021-05-04,121.360001,0.017619
2021-05-05,120.919998,0.016900
2021-05-06,122.919998,0.014956
2021-05-07,124.940002,0.013747
2021-05-10,126.750000,0.014062
2021-05-11,122.639999,0.012377
2021-05-12,117.620003,0.016296
2021-05-13,117.989998,0.020659


In [5]:
y = stock_df['Decision'].values.reshape(-1,1)
y[:20]

array([['Entry'],
       ['Exit'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Exit'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Entry'],
       ['Exit'],
       ['Entry'],
       ['Entry'],
       ['Exit'],
       ['Entry'],
       ['Entry'],
       ['Exit']], dtype=object)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)

In [8]:
predictions = model.predict(X_test_scaled)

In [9]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(
        n_estimators=1000,
        learning_rate=learning_rate,
        max_features=2,
        max_depth=3,
        random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.5f}".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.5f}".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.86708
Accuracy score (validation): 0.49387

Learning rate:  0.1
Accuracy score (training): 0.94632
Accuracy score (validation): 0.49233

Learning rate:  0.25
Accuracy score (training): 1.00000
Accuracy score (validation): 0.48160

Learning rate:  0.5
Accuracy score (training): 1.00000
Accuracy score (validation): 0.48313

Learning rate:  0.75
Accuracy score (training): 1.00000
Accuracy score (validation): 0.47239

Learning rate:  1
Accuracy score (training): 1.00000
Accuracy score (validation): 0.51994



In [10]:
predictions_df = pd.DataFrame({"Actual": y_test.ravel(),"Prediction": predictions}, index=stock_df.index[-len(y_test):])
predictions_df.head(20)

,Actual,Prediction
Date,,
2018-10-25,Entry,Entry
2018-10-26,Entry,Entry
2018-10-29,Exit,Entry
2018-10-30,Exit,Entry
2018-10-31,Exit,Exit
2018-11-01,Entry,Entry
2018-11-02,Entry,Exit
2018-11-05,Exit,Entry
2018-11-06,Entry,Entry


In [11]:
model = GradientBoostingClassifier(
    n_estimators=1000,
    learning_rate=1,
    max_features=2,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())

# Score the model
print("Accuracy score (training): {0:.5f}".format(
    model.score(
        X_train_scaled,
        y_train)))
print("Accuracy score (validation): {0:.5f}".format(
    model.score(
        X_test_scaled,
        y_test)))

Accuracy score (training): 1.00000
Accuracy score (validation): 0.51994


In [12]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

acc_score = accuracy_score(y_test, predictions)

In [13]:
print('Confusion Matrix')
display(cm_df)
print()
print()
print(f'Accuracy Score: {acc_score:.5f}')
print()
print()
print('Classification Report')
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,174,166
Actual Exit,173,139




Accuracy Score: 0.48006


Classification Report
              precision    recall  f1-score   support

       Entry       0.50      0.51      0.51       340
        Exit       0.46      0.45      0.45       312

    accuracy                           0.48       652
   macro avg       0.48      0.48      0.48       652
weighted avg       0.48      0.48      0.48       652



In [14]:
dot_data = tree.export_graphviz(
    model.estimators_[9, 0],
    out_file=None, filled=True,
    rounded=True,
    special_characters=True,
    proportion=True,
)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())


InvocationException: GraphViz's executables not found

## Random Oversampling

In [15]:
ros = RandomOverSampler(random_state=78)
X_over_resampled, y_over_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_over_resampled)

Counter({'Exit': 1043, 'Entry': 1043})

In [16]:
model_over_resample = LogisticRegression(solver='lbfgs', random_state=78)
model_over_resample.fit(X_over_resampled, y_over_resampled)
y_pred_over = model.predict(X_test_scaled)
cm_over = confusion_matrix(y_test, y_pred_over)
cm_over_df = pd.DataFrame(
    cm_over, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)
bac_over = balanced_accuracy_score(y_test, y_pred_over)

In [17]:
print('Confusion Matrix')
display(cm_over_df)
print()
print()
print(f'Balance Accuracy Score: {bac_over:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_over))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,192,148
Actual Exit,165,147




Balance Accuracy Score: 0.51793


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.54      0.56      0.47      0.55      0.52      0.27       340
       Exit       0.50      0.47      0.56      0.48      0.52      0.26       312

avg / total       0.52      0.52      0.52      0.52      0.52      0.27       652



## SMOTE

In [18]:
X_resampled_smote, y_resampled_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

Counter(y_resampled_smote)

Counter({'Exit': 1043, 'Entry': 1043})

In [19]:
model_smote = LogisticRegression(solver='lbfgs', random_state=78)
model_smote.fit(X_resampled_smote, y_resampled_smote)
y_pred_smote = model_smote.predict(X_test_scaled)
bac_smote = balanced_accuracy_score(y_test, y_pred_smote)
cm_smote = confusion_matrix(y_test, y_pred_smote)
cm_smote_df = pd.DataFrame(
    cm_smote, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [20]:
print('Confusion Matrix')
display(cm_smote_df)
print()
print()
print(f'Balance Accuracy Score: {bac_smote:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_smote))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,232,108
Actual Exit,224,88




Balance Accuracy Score: 0.48220


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.51      0.68      0.28      0.58      0.44      0.20       340
       Exit       0.45      0.28      0.68      0.35      0.44      0.18       312

avg / total       0.48      0.49      0.47      0.47      0.44      0.19       652



## Undersampling

In [21]:
rus = RandomUnderSampler(random_state=78)
X_under_resampled, y_under_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_under_resampled)

Counter({'Exit': 1043, 'Entry': 1043})

In [22]:
model_under_resample = LogisticRegression(solver='lbfgs', random_state=78)
model_under_resample.fit(X_under_resampled, y_under_resampled)
y_pred_under = model_under_resample.predict(X_test_scaled)
bac_under = balanced_accuracy_score(y_test, y_pred_under)
cm_under = confusion_matrix(y_test, y_pred_under)
cm_under_df = pd.DataFrame(
    cm_under, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [23]:
print('Confusion Matrix')
display(cm_under_df)
print()
print()
print(f'Balance Accuracy Score: {bac_under:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_under))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,234,106
Actual Exit,227,85




Balance Accuracy Score: 0.48034


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.51      0.69      0.27      0.58      0.43      0.20       340
       Exit       0.45      0.27      0.69      0.34      0.43      0.18       312

avg / total       0.48      0.49      0.47      0.47      0.43      0.19       652



## Cluster Centroid Undersampling

In [24]:
cluster = ClusterCentroids(random_state=78)
X_resampled_cc, y_resampled_cc = cluster.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_cc)

Counter({'Entry': 913, 'Exit': 913})

In [25]:
model_resample_cc = LogisticRegression(solver='lbfgs', random_state=78)
model_resample_cc.fit(X_resampled_cc, y_resampled_cc)
y_pred_cc = model_resample_cc.predict(X_test_scaled)
bac_cc = balanced_accuracy_score(y_test, y_pred_cc)
cm_cc = confusion_matrix(y_test, y_pred_cc)
cm_cc_df = pd.DataFrame(
    cm_cc, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [26]:
print('Confusion Matrix')
display(cm_cc_df)
print()
print()
print(f'Balance Accuracy Score: {bac_cc:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_cc))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,201,139
Actual Exit,187,125




Balance Accuracy Score: 0.49591


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.52      0.59      0.40      0.55      0.49      0.24       340
       Exit       0.47      0.40      0.59      0.43      0.49      0.23       312

avg / total       0.50      0.50      0.49      0.50      0.49      0.24       652



## SMOTEEN

In [27]:
sm = SMOTEENN(random_state=78)
X_resampled_smoteenn, y_resampled_smoteenn = sm.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_smoteenn)

Counter({'Entry': 161, 'Exit': 174})

In [28]:
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=78)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteenn)
y_pred_smoteenn = model_smoteenn.predict(X_test_scaled)
bac_smoteenn = balanced_accuracy_score(y_test, y_pred_smoteenn)
cm_smoteenn = confusion_matrix(y_test, y_pred_smoteenn)
cm_smoteenn_df = pd.DataFrame(
    cm_smoteenn, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

In [29]:
print('Confusion Matrix')
display(cm_smoteenn_df)
print()
print()
print(f'Balance Accuracy Score: {bac_smoteenn:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_smoteenn))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,87,253
Actual Exit,78,234




Balance Accuracy Score: 0.50294


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.53      0.26      0.75      0.34      0.44      0.18       340
       Exit       0.48      0.75      0.26      0.59      0.44      0.20       312

avg / total       0.50      0.49      0.51      0.46      0.44      0.19       652



## Balanced Random Forest Classifier

In [30]:
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=78)
brf.fit(X_train_scaled, y_train)
y_pred_brf = brf.predict(X_test_scaled)
bac_brf = balanced_accuracy_score(y_test, y_pred_brf)
cm_brf = confusion_matrix(y_test, y_pred_brf)
cm_brf_df = pd.DataFrame(
    cm_brf, index=["Actual Entry", "Actual Exit"], columns=["Predicted Entry", "Predicted Exit"]
)

C:\Users\Andy L\anaconda3\envs\alpacaenv\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [31]:
print('Confusion Matrix')
display(cm_brf_df)
print()
print()
print(f'Balance Accuracy Score: {bac_brf:.5f}')
print()
print()
print('Classification Report (Imbalanced)')
print(classification_report_imbalanced(y_test, y_pred_brf))

Confusion Matrix


,Predicted Entry,Predicted Exit
Actual Entry,161,179
Actual Exit,159,153




Balance Accuracy Score: 0.48196


Classification Report (Imbalanced)
                   pre       rec       spe        f1       geo       iba       sup

      Entry       0.50      0.47      0.49      0.49      0.48      0.23       340
       Exit       0.46      0.49      0.47      0.48      0.48      0.23       312

avg / total       0.48      0.48      0.48      0.48      0.48      0.23       652

